# Base model development

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
from model_class import CatBoostXT_BAG

## LOAD DATA

In [4]:
data_dir = r'D:\Documents\Fintech_master\Dissertation\Data'

In [5]:
df = pd.read_parquet(rf'{data_dir}\processed\final_df.parquet').reset_index(drop=True)
df = df.replace({None: np.nan})

In [6]:
df.head()

,SK_ID_CURR,AGE_YEARS,EMPLOY_YEARS,REG_YEARS_AGO,ID_PUBLISH_YEARS_AGO,PHONE_CHANGE_YEARS_AGO,CREDIT_TO_INCOME,ANNUITY_TO_INCOME,ANNUITY_TO_CREDIT,GOODS_TO_CREDIT,...,bb_w24_status_3_share__mean,bb_w24_status_3_share__max,bb_w24_status_3_share__sum,bb_w24_status_4_share__mean,bb_w24_status_4_share__max,bb_w24_status_4_share__sum,bb_w24_status_5_share__mean,bb_w24_status_5_share__max,bb_w24_status_5_share__sum,TARGET
0,100002,25.920548,1.745205,9.994521,5.808219,3.106849,2.007889,0.121978,0.060749,0.863262,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,100003,45.931507,3.254795,3.249315,0.797260,2.268493,4.790750,0.132217,0.027598,0.873211,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0
2,100004,52.180822,0.616438,11.671233,6.934247,2.232877,2.000000,0.100000,0.050000,1.000000,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0
3,100006,52.068493,8.326027,26.939726,6.676712,1.690411,2.316167,0.219900,0.094941,0.949845,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,100007,54.608219,8.323288,11.810959,9.473973,3.030137,4.222222,0.179963,0.042623,1.000000,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,0


In [7]:
# separate features, target
list_features = [col for col in df.columns if col not in ['TARGET','SK_ID_CURR']]
list_key = ['SK_ID_CURR']
label_col = 'TARGET'

In [8]:
list_cat_features = df.select_dtypes(include=['object','category']).columns.tolist()
list_cat_features

['w30_last_loan_type',
 'w90_last_loan_type',
 'w180_last_loan_type',
 'w360_last_loan_type',
 'w720_last_loan_type']

In [16]:
df[list_cat_features] = df[list_cat_features].fillna('Missing')

In [17]:
len(list_features)

476

In [18]:
# split train test
X_train, X_test, y_train, y_test = train_test_split(df[list_features], df[label_col], test_size=0.25, random_state=1, stratify=df[label_col])

In [19]:
y_train

226530    0
130214    0
91848     0
249063    0
20697     0
         ..
276129    1
252261    0
215416    0
88067     0
232520    1
Name: TARGET, Length: 230633, dtype: int64

In [20]:
# summary of train test split data, count of target variable
print('Train set shape:', X_train.shape, y_train.shape)
print(y_train.value_counts(normalize=True))
print(y_train.value_counts())

print('Test set shape:', X_test.shape, y_test.shape)
print(y_test.value_counts(normalize=True))
print(y_test.value_counts())

Train set shape: (230633, 476) (230633,)
TARGET
0    0.91927
1    0.08073
Name: proportion, dtype: float64
TARGET
0    212014
1     18619
Name: count, dtype: int64
Test set shape: (76878, 476) (76878,)
TARGET
0    0.919275
1    0.080725
Name: proportion, dtype: float64
TARGET
0    70672
1     6206
Name: count, dtype: int64


In [33]:
base_model = CatBoostXT_BAG(
    num_bag_folds=5,
    num_bag_repeats=2,
    random_state=1,
    cat_features=list_cat_features
)

In [34]:
base_model.params = {
    'iterations': 1000,
    'learning_rate': 0.025,
    'depth': 4,
    'l2_leaf_reg': 25,
    'border_count': 64,
    'auto_class_weights': 'SqrtBalanced',
    'early_stopping_rounds': None,
    'od_type': 'IncToDec',
    'od_wait': 100,
    'od_pval': 0.001,
    'boosting_type': 'Ordered',
    'bootstrap_type': 'MVS',
}

In [35]:
base_model.fit(X_train, y_train)

MemoryError: Unable to allocate 655. MiB for an array with shape (465, 184506) and data type float64